In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
#install Neptune client
!pip install neptune-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.6/456.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


# Importing Configurations and Hyperparameters

In [ ]:
train_path = "/content/drive/My Drive/mashqa_data/train_webmd_squad_v2_consec.json"
val_path = "/content/drive/My Drive/mashqa_data/val_webmd_squad_v2_consec.json"
test_path = "/content/drive/My Drive/mashqa_data/test_webmd_squad_v2_consec.json"

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
#dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "llama-2-7b-miniguanaco"

# Load the model and define necessary parameters ready for modeling

In [ ]:
from torch.utils.data import Dataset
import torch

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, questions, answers, max_length):
        self.tokenizer = tokenizer
        self.questions = questions
        self.answers = answers
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]
        encoding = self.tokenizer(question, answer, truncation=True, padding='max_length', max_length=self.max_length)
        return {key: torch.tensor(val) for key, val in encoding.items()}

In [ ]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

In [ ]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

In [ ]:
output_dir = "/content/drive/MyDrive/MSCA32021FinalProject"
num_train_epochs = 0.2 #change this
fp16 = False
bf16 = False
per_device_train_batch_size = 4 #change this
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1 #change this
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 200 #change this
logging_steps = 200 #change this


In [ ]:
max_seq_length = None
packing = False
device_map = {"": 0}


In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Import the base Model and tokenizer

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Introduce Platform and Record the Response

In [ ]:
import neptune.new as neptune

/usr/local/lib/python3.10/dist-packages/neptune/internal/backends/hosted_client.py:51: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
<ipython-input-22-37c521bab1e7>:1: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [ ]:
run=neptune.init_run(
    project="a657774956/MlopsFinal",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NDQyMjgxYi01NDQzLTRlZDQtODA1ZS1mNDY1ZWM4M2UxM2QifQ==",
)

<ipython-input-23-a4d151eb1ef6>:1: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run=neptune.init_run(


https://app.neptune.ai/a657774956/MlopsFinal/e/MLOP-2


In [ ]:
from transformers import AutoTokenizer, pipeline
#import logging

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Define the prompts
prompts = [
    "What types of exercise are best for people with asthma?",
    "How is obsessive-compulsive disorder diagnosed?",
    "When are you more likely to get a blood clot?",
    "How should you lift objects to prevent back pain?",
    "How can you be smart with antibiotics?",
]

# Assuming 'model' variable is defined earlier and contains the desired model
tokenizer = AutoTokenizer.from_pretrained(model.name_or_path)

# Run text generation pipeline with the specified model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)

# Iterate through the prompts and print the results
for m, prompt in enumerate(prompts):
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    generated_text = result[0]['generated_text']
    answer = generated_text.split('[INST]')[-1]  # Extract the answer part after the '[INST]' tag

    # Log the question and answer to Neptune
    run[f'base_model/qa_pair_{m}/question'] = prompt
    run[f'base_model/qa_pair_{m}/answer'] = answer

    print(f"Question: {prompt}\nAnswer: {answer}\n{'-'*50}\n")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Question: What types of exercise are best for people with asthma?
Answer:  What types of exercise are best for people with asthma? [/INST]  Exercise is an important part of overall health and wellness, and for people with asthma, it can be beneficial for improving lung function and overall health. everybody. However, it's important to exercise safely and effectively, especially for people with asthma. Here are some types of exercise that are best for people with asthma:

1. Aerobic exercise: Aerobic exercise, such as brisk walking, jogging, cycling, or swimming, can help improve lung function and overall cardiovascular health. These types of exercises are generally considered safe for people with asthma and can help reduce symptoms.
2. Yoga and Pilates: These forms of exercise focus on strengthening the core and improving flexibility, which can help improve lung function and overall breathing. Many people with asthma find these exercises to be helpful in managing their symptoms.
3. Res

# Processing the data by extracting questions and answers

In [ ]:
import json

def load_data(path):
    with open(path, 'r') as file:
        data = json.load(file)
    # Process data as needed for your specific structure
    return data

train_data = load_data(train_path)
val_data = load_data(val_path)
# Process other files as needed


In [ ]:
questions_and_answers = {'questions': [], 'answers': []}

for m in range(len(train_data['data'])):
  for paragraph in train_data['data'][m]['paragraphs']:
    for qas in paragraph['qas']:
      question = qas['question']
      answer = qas['answers'][0]['text']  # Assuming there is always at least one answer

        # Add to our dictionary
      questions_and_answers['questions'].append(question)
      questions_and_answers['answers'].append(answer)


In [ ]:
val_questions_and_answers = {'questions': [], 'answers': []}

for m in range(len(val_data['data'])):
  for paragraph in val_data['data'][m]['paragraphs']:
    for qas in paragraph['qas']:
      question = qas['question']
      answer = qas['answers'][0]['text']  # Assuming there is always at least one answer

      # Add to our dictionary
      val_questions_and_answers['questions'].append(question)
      val_questions_and_answers['answers'].append(answer)


# Further adjust parameters to make the base model ready for training

In [ ]:
max_length = 128 # Define the maximum sequence length
train_dataset = CustomDataset(tokenizer, questions_and_answers['questions'], questions_and_answers['answers'], max_length)

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


# Train the model

In [ ]:
# Train model
trainer.train()

{'loss': 1.7034, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 1.5865, 'learning_rate': 0.0002, 'epoch': 0.08}
{'loss': 1.5485, 'learning_rate': 0.0002, 'epoch': 0.12}
{'loss': 1.5372, 'learning_rate': 0.0002, 'epoch': 0.16}
{'loss': 1.5291, 'learning_rate': 0.0002, 'epoch': 0.2}
{'train_runtime': 616.4018, 'train_samples_per_second': 6.486, 'train_steps_per_second': 1.622, 'train_loss': 1.580924530029297, 'epoch': 0.2}


TrainOutput(global_step=1000, training_loss=1.580924530029297, metrics={'train_runtime': 616.4018, 'train_samples_per_second': 6.486, 'train_steps_per_second': 1.622, 'train_loss': 1.580924530029297, 'epoch': 0.2})

# Export the trained Model and records the response to the platform

In [ ]:
new_model=trainer.model
tokenizer = AutoTokenizer.from_pretrained(new_model.name_or_path)

# Run text generation pipeline with the newly trained model
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=1000)


In [ ]:
# Iterate through the prompts and print the results
for m, prompt in enumerate(prompts):
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    generated_text = result[0]['generated_text']
    answer = generated_text.split('[INST]')[-1]  # Extract the answer part after the '[INST]' tag

    # Log the question and answer to Neptune for the newly trained model
    run[f'trained_model/qa_pair_{m}/question'] = prompt
    run[f'trained_model/qa_pair_{m}/answer'] = answer

    print(f"Question: {prompt}\nAnswer: {answer}\n{'-'*50}\n")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Question: What types of exercise are best for people with asthma?
Answer:  What types of exercise are best for people with asthma? [/INST]  Exercise is a great way to improve your health and asthma symptoms. everybody needs to exercise, but people with asthma need to be careful. Some types of exercise can trigger asthma symptoms. Others can help you control them. Here are some types of exercise that are best for people with asthma: Swimming: Swimming is a great way to get exercise without triggering asthma symptoms. It's also a low-impact exercise, which means it's easier on your joints. It's also a great way to get your heart rate up without having to run. Cycling: Cycling is another low-impact exercise that can help you get your heart rate up without triggering asthma symptoms. It's also a great way to get some exercise without having to go to a gym. Walking: Walking is a great way to get some exercise without having to go to a gym. It's also a low-impact exercise, which means it's e